# Working With `ParseTree` Objects
`ParseTree` objects are compact data structures for storing parsetree information. They can be used like sequences or trees of `Token` objects (see more info abou that below). [Check the docs](ref/doctable.parsetree.html) for object details. I'll try to add some explanations below.


### `Token` Objects
Each token in a `ParseTree` is represented by a `Token` object. These objects maintain the tree structure of a parsetree, and each node contains some default information as well as optional and custom information. These are the most important member variables.

#### Member Variables
+ _parent_: reference to parent node
+ _childs_: list of references to child nodes
+ _i_: index of token in sentence
+ _text_: text representation of token
+ _tag_: the part-of-speech tag offered by the dependency parser (different from POS tagger)
+ _dep_: the dependency relation to parent object. See the [Spacy annotation docs](https://spacy.io/api/annotation#dependency-parsing) for more detail.

#### Optional Member Variables
+ _info_: any additional info generated when creating parsetree
+ _pos()_: part-of-speech tag created if user enablled POS 'tagger' in Spacy. See [Spacy POS tag docs](https://spacy.io/api/annotation#pos-tagging) for more detail. Also check out docs for [UPOS tags](https://universaldependencies.org/docs/u/pos/).
+ _ent()_: named entity type of token (if NER was enabled when creating parsetree). See [Spacy NER docs](https://spacy.io/api/annotation#named-entities) for more detail.

In [1]:
example_text = 'I’ve outrun Imperial starships.'

In [2]:
# import doctable
import sys
sys.path.append('..')
import doctable

We now create a parsetree by passing the Document object output fromthe spacy parser (called `nlp` here) to the `get_parsetrees` function.

In [3]:
import spacy
from spacy import displacy
nlp = spacy.load('en')
doc = nlp(example_text)
print(doc)
displacy.render(doc, style="dep", jupyter=True, options={'compact':False, 'distance':100})

I’ve outrun Imperial starships.


In [4]:
ptree = doctable.get_parsetrees(doc, parse_tok_func=doctable.parse_tok)[0]
ptree.display(pad=18, base=5) # the shorter parsetree in this example

’ve   -nsubj> i        
      -dobj> starships  -compound> outrun
                        -compound> Imperial
                        -punct> .        
     

## Using Iteration
We can easily iterate through the tokens in the tree sequentially.

In [5]:
for tok in ptree:
    print(tok.i, tok)

0 Token(i)
1 Token(’ve)
2 Token(outrun)
3 Token(Imperial)
4 Token(starships)
5 Token(.)


In [6]:
# we can also convert to dictionary with .as_dict()
for tok in ptree:
    print('{i}: {text} ({tag})'.format(**tok.as_dict()))

0: i (PRP)
1: ’ve (VB)
2: outrun (VBN)
3: Imperial (JJ)
4: starships (NNS)
5: . (.)


## Using Recursion
Recursion is another useful way to navigate parsetrees. This approach will allow you to move through the tree as a structure. You can always get the root of a parse tree from `root`, and children of each node using childs.

In [7]:
ptree.root

Token(’ve)

In [8]:
ptree.root.childs

[Token(i), Token(starships), Token(.)]

The example below creates a function `print_recursion` which simply demonstrates how recursion works on parsetrees.

In [9]:
def print_recursion(tok, level=0):
    if not tok.childs:
        print('    '*level + 'base node', tok)
    else:
        print('    '*level + 'entering', tok)
        for child in tok.childs:
            print_recursion(child, level+1)
        print('    '*level + 'leaving', tok)

print_recursion(ptree.root)

entering Token(’ve)
    base node Token(i)
    entering Token(starships)
        base node Token(outrun)
        base node Token(Imperial)
    leaving Token(starships)
    base node Token(.)
leaving Token(’ve)
